In [5]:
import pandas as pd
from importlib import reload
import sklearn as sk

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.min_rows', 999)

%config IPCompleter.use_jedi = False

In [7]:
df = pd.read_csv("./df_with_counties_v1.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,district_cd,cohortye,total,gr_droppedout,gr_graduated,gr_iep,gr_ged,gr_stillenrolled,outcome_yrs,srcyear,subgroup,_name_,gr_local_cnt,year,district_from_df1,num_asian,num_black,num_hisp,num_am_ind,num_white,num_female,num_male,num_lep,num_free_lunch,num_reduced_lunch,num_multi,num_swd,num_ecdis,num_ell,district_from_faru,ufb,staid,star,totstat,lrev,fedrev,totrev,state_aid_total_revenues,star_total_revenues,state_revenues_total_revenues,bded,cadm,tsal,pps,cds,boc,tui1,tui2,ois,oiexp,cser,opmt,tchr,heal,oeb,total_fringe_benefits,ound,oth,subtot,trans,dsp,dsi,texp,total_unexpended_surplus_funds_t,revenue_expenditures,var,dcaadm,rev_pupil,exp_pupil,av_twpu,inc_twpu,pwr,apwr,cwr,instructional_salaries,instructional_fringe_benefits,ins_fb,adjusted_expenditures__subtotal,ie1,ie1_total,ie2_instructional_expenditures_i,ie2,ie3_instructional_expenditures_i,ie3_adj_exp,loc_eff_rate,districtid,name,county_name,district_name
0,0,0,10100,2000,618,83.0,407,21.0,57.0,50.0,5.0,200506,1.0,NaN,NaN,2005,10100,269.0,5952.0,918.0,21.0,1884.0,NaN,NaN,300.0,5505.0,970.0,0.0,NaN,NaN,NaN,10100,5680290,54570126,10225351,64795477,98285968,16659701,179741146,0.303604,0.056889,0.360493,490999,2118256,54023846,5251097,4501195,3728010,188593,6610228,16305802,19303934,1199428,11203660,5165294,14257866,14441655,33864815,4053261,564400,163407524,5972182,7669294,5365766,182414766,0.031139,-2673620,-0.014657,10559,17022.553840,17275.761530,244659,93587,0.706290,0.789764,0.748027,83809950,3.117730e+07,0.170914,162986931,109912705.0,0.602543,1.410900e+08,0.773457,1.409014e+08,0.864495,26.51,10100.0,albany city,Albany,albany city
1,1,1,10100,2001,579,100.0,384,40.0,27.0,28.0,5.0,200706,1.0,NaN,NaN,2007,10100,360.0,5281.0,940.0,24.0,1654.0,NaN,NaN,382.0,3992.0,789.0,77.0,NaN,NaN,NaN,10100,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,10100.0,albany city,Albany,albany city
2,2,2,10100,2001,579,84.0,327,30.0,26.0,112.0,4.0,200706,1.0,NaN,NaN,2007,10100,360.0,5281.0,940.0,24.0,1654.0,NaN,NaN,382.0,3992.0,789.0,77.0,NaN,NaN,NaN,10100,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,10100.0,albany city,Albany,albany city
3,3,3,10100,2001,579,103.0,397,41.0,29.0,9.0,6.0,200706,1.0,NaN,NaN,2007,10100,360.0,5281.0,940.0,24.0,1654.0,NaN,NaN,382.0,3992.0,789.0,77.0,NaN,NaN,NaN,10100,13358464,74675977,10131780,84807757,105784891,13677063,204269711,0.365575,0.049600,0.415175,385910,2055778,55114952,5988074,4266372,4793593,541340,7253219,17557992,26611527,602434,11451223,5439024,16966397,11086334,33491755,3264726,989999,174368894,6153173,7356028,6733231,194611326,0.068642,9658385,0.049629,10430,19584.823682,18658.804027,268181,99117,0.628353,0.725600,0.676977,87720983,3.087779e+07,0.158664,173595554,122127069.0,0.627543,1.530049e+08,0.786207,1.524635e+08,0.878269,21.95,10100.0,albany city,Albany,albany city
4,4,4,10100,2002,602,143.0,389,22.0,15.0,33.0,5.0,200806,1.0,NaN,NaN,2008,10100,382.0,4894.0,908.0,27.0,1592.0,NaN,NaN,405.0,4660.0,757.0,96.0,NaN,NaN,NaN,10100,11443927,88986207,9683819,98670026,108582740,12186499,219439265,0.405516,0.044130,0.449646,325611,2333872,57538745,632

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

X = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'gr_droppedout', 'gr_graduated', 'gr_iep', 'gr_ged', 'gr_stillenrolled', '_name_', 'district_from_df1', 'district_from_faru', 'districtid'])
y_grad = df.gr_graduated

X_train, X_test, y_train, y_test = train_test_split(X, y_grad, test_size=0.25, random_state=42)

In [9]:
from sklearn.compose import ColumnTransformer

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, categorical_features)])

rf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier())])

In [10]:
rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['district_cd', 'cohortye', 'total', 'outcome_yrs', 'srcyear',
       'subgroup', 'gr_local_cnt', 'year', 'num_asian', 'num_black',
       'num_hisp', 'num_am_ind', 'num_white', 'num_female', 'num_male',
       'num_le...
       'ie2_instructional_expenditures_i', 'ie2',
       'ie3_instructional_expenditures_i', 'ie3_adj_exp', 'loc_eff_rate'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
   

In [11]:
%%time
y_pred = rf.predict(X_train)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, y_pred)

In [30]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
    
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

KNeighborsClassifier(n_neighbors=3)
model score: 0.217


In [ ]:
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']}from sklearn.model_selection import GridSearchCVCV = GridSearchCV(rf, param_grid, n_jobs= 1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)